In [ ]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session.

In [ ]:
import torch
from torch import nn

torch.__version__

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
import requests
import zipfile

from pathlib import Path

data_path=Path("data/")
image_path=data_path / "pizza_steak_sushi"

image_path.mkdir(parents=True, exist_ok=True)
    
with open(data_path / "pizza_steak_sushi.zip","wb") as f:
    request=requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    f.write(request.content)
        
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip","r")as zip_ref:
    zip_ref.extractall(image_path)

In [ ]:
import os
def walk_through_dir(dir_path):
#     pass
    for dirpath, dirname, filenames in os.walk(dir_path):
        print(len(dirname), len(filenames), dirpath)

In [ ]:
walk_through_dir(image_path)

In [ ]:
train_dir=image_path / "train"
test_dir=image_path / "test"

train_dir, test_dir

In [ ]:
import random
from PIL import Image

random.seed(42)

# get all image paths (* means "any combination")
image_path_list=list(image_path.glob("*/*/*.jpg"))

# get random image path
random_img_path=random.choice(image_path_list)

# get image class fro path name
image_class=random_img_path.parent.stem

# open image
img=Image.open(random_img_path)

print(random_img_path)
print(image_class)
print(img.height)
print(img.width)
img

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
data_transform=transforms.Compose([
    
    #resize the images to 64x64
    transforms.Resize(size=(64,64)),
    
    # flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p= probability of flip
    
    # turn the image into a torch.Tensor
    transforms.ToTensor() # this aslo convert all pixel value from 0to 255 to be between 0 and 1
])

In [ ]:
import matplotlib.pyplot as plt

img_as_array=np.asarray(img)

plt.figure(figsize=(10,7))
plt.imshow(img_as_array)

In [ ]:
def plot_transformed_images(image_paths, transform, n=3, seed=29):
    random.seed(seed)
    random_image_paths=random.sample(image_paths, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax=plt.subplots(1,2)
            ax[0].imshow(f)
            ax[0].set_title(f.size)
            ax[0].axis("off")
            
            # trasnform and plot mage
            # permute() will change shape of image to suit matplotlib
            # pytorch default is [C,H,W] But matplotlib is [H,W,C]
            transformed_image=transform(f).permute(1,2,0)
            ax[1].imshow(transformed_image)
            ax[1].set_title(transformed_image.shape)
            ax[1].axis(False)
            
            fig.suptitle(image_path.parent.stem,fontsize=16)
            
plot_transformed_images(image_path_list,
                        transform=data_transform,
                        n=3)

In [ ]:
# use ImageFolder to create datasets
train_data=datasets.ImageFolder(root=train_dir, # target folderof image
                                transform=data_transform, # 
                                target_transform=None)# transforms to perform on label (if necesary)
test_data=datasets.ImageFolder(root=test_dir,
                               transform=data_transform)
print(train_data,test_data)

In [ ]:
class_names=train_data.classes
class_names

In [ ]:
class_dict=train_data.class_to_idx
class_dict

In [ ]:
img, label=train_data[0][0], train_data[0][1]
print(img, img.shape)
print("\n")

print(img.dtype)
print("\n")
print(label, type(label))

In [ ]:
# rearrange the order of dimensions
img_permute=img.permute(1,2,0)

# print out different shapes (before and after permute)
print(f"{img.shape} ->[color_channels,h,w]")
print(f"{img_permute.shape} -> [h,w,color_channels]")

plt.figure(figsize=(10,7))
plt.imshow(img.permute(1,2,0))
# plt.axis(False)
plt.title(class_names[label])

In [ ]:
# tur n train adn ttest datasets inot dataloaders
from torch.utils.data import DataLoader

train_dataloader=DataLoader(dataset=train_data,
                            batch_size=1,
                            num_workers=1, # how many suprocess to usefor data loading?
                            shuffle=True)
test_dataloader=DataLoader(dataset=test_data,
                           batch_size=1,
                           num_workers=1,
                           shuffle=False)
train_dataloader,test_dataloader

In [ ]:
img, label=next(iter(train_dataloader))

print(f"{img.shape} -> [batch_size,cc,h,w]")
print(label.shape)

In [ ]:
import os
import pathlib
import torch

from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from typing import Tuple, Dict, List

In [ ]:
train_data.classes, train_data.class_to_idx

In [ ]:
# setup path for target directiory
target_directory=train_dir
print(target_directory)

class_names_found=sorted(entry.name for entry in list(os.scandir(image_path / "train")))
print(class_names_found)

In [ ]:
def find_classes(directory)-> Tuple[List[str], Dict[str,int]]:
    # get the classes by scanning the target directory
    classes=sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
    
    # raise an error if class names not found
    if not classes:
        raise FileNotFoundError(f"couldn't find any classes in {directory}")
    
    # create a directory of index labels (computer prefer numerical rather than string labels)
    class_to_idx={cls_name:i for i, cls_name in enumerate(classes) }
    return classes, class_to_idx

In [ ]:
find_classes(train_dir)

In [ ]:
from torch.utils.data import Dataset

# subclass torch.utils.data.Dataset
class ImageFolderCustom(Dataset):

    # initialize with a targ_dir and transform parameter
    def __init__(self,targ_dir:str, transform=None)->None:
        # create class attributes 
        
        # get all image paths
        self.paths=list(pathlib.Path(targ_dir).glob("*/*.jpg"))
        
        # setup transforms
        self.transform=transform
        
        self.classes, self.classes_to_idx=find_classes(targ_dir)
        
    def load_image(self,index:int)->Image.Image:
        image_path=self.paths[index]
        return Image.open(image_path)
    
    # overwrite the __len__() method (optional but recommended fro subclasses of torch.utils.data.Dataset)
    def __len__(self)->int:
        return len(self.paths)
    
    # overwrite the __getiitem__() method (required fro subclasses of torch.utils.data.Dataset)
    def __getitem__(self,index:int)->Tuple[torch.Tensor,int]:
        img=self.load_image(index)
        class_name=self.paths[index].parent.name
        class_idx=self.classes_to_idx[class_name]
        
        if self.transform:
            return self.transform(img), class_idx
        else:
            return img, clas_idx

In [ ]:
# augment train data
train_transforms=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

# don't augmenyt test data only reshape
test_transforms=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

In [ ]:
train_data_custom=ImageFolderCustom(targ_dir=train_dir,
                                   transform=train_transforms)
test_data_custom=ImageFolderCustom(targ_dir=test_dir,
                                   transform=test_transforms)

train_data_custom,test_data_custom

In [ ]:
len(train_data_custom),len(test_data_custom)

In [ ]:
train_data_custom.classes,train_data_custom.classes_to_idx

In [ ]:
print((len(train_data_custom)==len(train_data)) & (len(test_data_custom)==len(test_data)))

In [ ]:
def display_random_images(dataset:torch.utils.data.dataset.Dataset,
                          classes:List[str]=None,
                          n:int=10,
                          display_shape:bool=True,
                          seed:int=None):
    if n>10:
        n=10
        display_shape=False
        print("n shouldn't be larger than 10")
    
    if seed:
        random.seed(seed)
    
    random_samples_idx=random.sample(range(len(dataset)),k=n)
    
    for i, targ_sample in enumerate(random_samples_idx):
        targ_image, targ_label=dataset[targ_sample][0], dataset[targ_sample][1]
        
        targ_image_adjust=targ_image.permute(1,2,0)
        
        plt.subplot(1,n,i+1)
        plt.imshow(targ_image_adjust)
        plt.axis("off")
        if classes:
            title=f"class: {classes[targ_label]}"
            if display_shape:
                title=title+f"\nshape{targ_image_adjust.shape}"
        plt.title(title)

In [ ]:
display_random_images(train_data,
                      n=5,
                      classes=class_names,
                      seed=None)

In [ ]:
display_random_images(train_data,
                      n=11,
                      classes=class_names,
                      seed=None)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader_custom=DataLoader(dataset=train_data_custom,
                                   batch_size=1,
                                   num_workers=0,
                                   shuffle=True)
test_dataloader_custom=DataLoader(dataset=test_data_custom,
                                  batch_size=1,
                                  num_workers=0,
                                  shuffle=False)

train_dataloader,test_dataloader

In [ ]:
img, label = next(iter(train_dataloader_custom))

print(f"{img.shape}->[batch,cc,h,w]")
print(f"{label.shape}")

In [ ]:
tain_transforms=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])
test_transforms=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [ ]:
image_path_list=list(image_path.glob("*/*/*.jpg"))

plot_transformed_images(
    image_paths=image_path_list,
    transform=train_transforms,
    n=3,
    seed=None
)

In [ ]:
simple_transform=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

In [ ]:
from torchvision import datasets
train_data_simple=datasets.ImageFolder(root=train_dir,
                                       transform=simple_transform,
                                       )
test_data_simple=datasets.ImageFolder(root=test_dir,
                                      transform=simple_transform)

import os 
from torch.utils.data import DataLoader
BATCH_SIZE=32
NUM_WORKERS=os.cpu_count()
print(BATCH_SIZE,NUM_WORKERS)

train_dataloader_simple=DataLoader(train_data_simple,
                                   batch_size=BATCH_SIZE,
                                   shuffle=True,
                                   num_workers=NUM_WORKERS)
test_dataloader_simple=DataLoader(test_data_simple,
                                 batch_size=BATCH_SIZE,
                                 shuffle=False,
                                 num_workers=NUM_WORKERS)

train_dataloader_simple,test_dataloader_simple

In [ ]:
class TinyVGG(nn.Module):
    def __init__(self,input_shape:int,hidden_units:int, output_shape):
        super().__init__()
        self.conv_block1=nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.conv_block2=nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*15*15,out_features=output_shape)
        )
    def forward(self,x:torch.Tensor):
            x=self.conv_block1(x)
            x=self.conv_block2(x)
            print(x.shape)
            x=self.classifier(x)
            return x
        
torch.manual_seed(29)
model_0=TinyVGG(3,10,len(train_data.classes)).to(device)
model_0

In [ ]:
img_batch, label_batch=next(iter(train_dataloader_simple))

img_single, label_single=img_batch[0].unsqueeze(dim=0), label_batch[0]
print(img_single.shape)

model_0.eval()
with torch.inference_mode():
    pred=model_0(img_single.to(device))

print("\n")
print(pred)
print(torch.softmax(pred,dim=1))
print(torch.argmax(torch.softmax(pred,dim=1),dim=1))
print(label_single)

In [ ]:
try:
    import torchinfo
except:
    !pip install torchinfo
    import torchinfo

In [ ]:
from torchinfo import summary
summary(model_0,input_size=[1,3,64,64])

In [ ]:
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer):
    model.train()
    
    train_loss, train_acc=0,0
    
    for batch, (X,y) in enumerate(dataloader):
        X,y=X.to(device), y.to(device)
        
        y_pred=model(X)
        
        loss=loss_fn(y_pred,y)
        train_loss+=loss.item()
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
        train_acc+=(y_pred_class==y).sum().item()/len(y_pred)
    
    # adjust metrics to get average loss and accuracy pre batch
    train_loss=train_loss / len(dataloader)
    train_acc=train_acc/ len(dataloader)
    
    return train_loss, train_acc

In [ ]:
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module):
    
    model.eval()
    
    test_loss, test_acc=0,0
    
    with torch.inference_mode():
        for batch, (X,y) in enumerate(dataloader):
            X,y=X.to(device), y.to(device)
            
            test_pred_logits=model(X)
            
            loss=loss_fn(test_pred_logits,y)
            test_loss+=loss.item()
            
            test_pred_labels=test_pred_logits.argmax(dim=1)
            test_acc+=((test_pred_labels==y).sum().item() / len(test_pred_labels))
            
    test_loss/=len(dataloader)
    test_acc/=len(dataloader)
    return test_loss, test_acc

In [ ]:
from tqdm.auto import tqdm

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int=5):
    results={"train_loss":[],
             "train_acc":[],
             "test_loss":[],
             "test_acc":[]}
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc=train_step(model=model,
                                         dataloader=train_dataloader,
                                         loss_fn=loss_fn,
                                         optimizer=optimizer)
        test_loss, test_acc=test_step(model=model,
                                      dataloader=test_dataloader,
                                      loss_fn=loss_fn)
        
        print(
            f"epoch+1 | " 
            f"train_loss: {train_loss} | "
            f"train_acc: {train_acc} | "            
            f"test_loss: {test_loss} | "
            f"test_acc: {test_acc} "
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
    return results

In [ ]:
torch.manual_seed(29)
torch.cuda.manual_seed(29)

NUM_EPOCHS=5
model_0=TinyVGG(input_shape=3,
                hidden_units=10,
                output_shape=len(train_data.classes)).to(device)

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params=model_0.parameters(),lr=0.01)

from timeit import default_timer as timer
start_time=timer()

model_0_res=train(model=model_0,
                  train_dataloader=train_dataloader_simple,
                  test_dataloader=test_dataloader_simple,
                  optimizer=optimizer,
                  loss_fn=loss_fn,
                  epochs=NUM_EPOCHS)

end_time=timer()
print(end_time - start_time)

In [ ]:
def plot_loss_curves(results:Dict[str,List[float]]):
    loss=results["train_loss"]
    test_loss=results["test_loss"]
    
    accuracy=results["train_acc"]
    test_accuracy=results["test_acc"]
    
    epochs=range(len(results["train_loss"]))
    
    plt.figure(figsize=(15,7))
    
    plt.subplot(1,2,1)
    plt.plot(epochs,loss,label="train_loss")
    plt.plot(epochs,test_loss,label="test_loss")
    plt.legend()
    
    plt.subplot(1,2,2)
    plt.plot(epochs,accuracy,label="train_acc")
    plt.plot(epochs,test_accuracy,label="test_acc")
    plt.legend()

In [ ]:
plot_loss_curves(model_0_res)

In [ ]:
train_transform_trivial_augment=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

test_transform=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

In [ ]:
train_data_augmented=datasets.ImageFolder(train_dir,transform=train_transform_trivial_augment)
test_data_simple=datasets.ImageFolder(test_dir,transform=test_transform)

train_data_augmented,test_data_simple

In [ ]:
import os
BATCH_SIZE=32
NUM_WORKERS=os.cpu_count()

torch.manual_seed(29)

train_dataloader_augmented=DataLoader(train_data_augmented,
                                      batch_size=BATCH_SIZE,
                                      shuffle=True,
                                      num_workers=NUM_WORKERS)

test_dataloader_simple=DataLoader(test_data_simple,
                                  batch_size=BATCH_SIZE,
                                  shuffle=False,
                                  num_workers=NUM_WORKERS)

train_dataloader_augmented, test_dataloader_simple

In [ ]:
torch.manual_seed(42)
model_1=TinyVGG(
    input_shape=3,
    hidden_units=10,
    output_shape=len(train_data_augmented.classes)
).to(device)
model_1

In [ ]:
torch.manual_seed(29)
torch.cuda.manual_seed(29)

NUM_EPOCHS=5

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model_1.parameters(),0.01)

from timeit import default_timer as timer
start_time=timer()

model_1_res=train(model_1,
                  train_dataloader_augmented,
                  test_dataloader_simple,
                  optimizer,
                  loss_fn,
                  NUM_EPOCHS)

end_time=timer()
print(end_time-start_time)

In [ ]:
plot_loss_curves(model_1_res)

In [ ]:
model_0_df=pd.DataFrame(model_0_res)
model_1_df=pd.DataFrame(model_1_res)
model_0_df

In [ ]:
epochs=range(len(model_0_df))

plt.subplot(2,2,1)
plt.plot(epochs,model_0_df["train_loss"],label="model_0")
plt.legend()

plt.subplot(2,2,2)
plt.plot(epochs,model_1_df["train_loss"],label="model_1")
plt.legend()

plt.subplot(2,2,3)
plt.plot(epochs,model_0_df["train_acc"],label="model_0")
plt.plot(epochs,model_1_df["train_acc"],label="model_1")
plt.legend()

plt.subplot(2,2,4)
plt.plot(epochs,model_0_df["test_acc"],label="model_0")
plt.plot(epochs,model_1_df["test_acc"],label="model_1")
plt.legend()

In [ ]:
import requests

custom_image_path=data_path / "04-pizza-dad.jpeg"

if not custom_image_path.is_file():
    with open(custom_image_path,"wb") as f:
        request=requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/04-pizza-dad.jpeg")
        f.write(request.content)
        
    
else:
    print("exists")


In [ ]:
import torchvision

custom_image_uint8=torchvision.io.read_image(str(custom_image_path))

print(custom_image_uint8)
print(custom_image_uint8.shape)
print(custom_image_uint8.dtype)

In [ ]:
# try to make prediction on image inuint8 format (this will error)
model_1.eval()
with torch.inference_mode():
    model_1(custom_image_uint8.to(device))

In [ ]:
custom_image=torchvision.io.read_image(str(custom_image_path)).type(torch.float32)

custom_image=custom_image / 255.

print(custom_image)
print(custom_image.shape)
print(custom_image.dtype)

In [ ]:
plt.imshow(custom_image.permute(1,2,0)) # CHW to HWC  fro matplotlib

In [ ]:
custom_image_transform=transforms.Compose([
    transforms.Resize((64,64))
])

custom_image_transformed=custom_image_transform(custom_image)

custom_image.shape, custom_image_transformed.shape

In [ ]:
model_1.eval()
with torch.inference_mode():
    custom_image_pred=model_1(custom_image_transformed)

In [ ]:
model_1.eval()
with torch.inference_mode():
    custom_image_pred=model_1(custom_image_transformed.to(device))

In [ ]:
model_1.eval()
with torch.inference_mode():
    
    # add an extra dimension  to imagae   -> BCHW batch size
    custom_image_transformed_with_batch_size=custom_image_transformed.unsqueeze(dim=0)
    
    print(custom_image_transformed.shape)
    print(custom_image_transformed_with_batch_size.shape)
    
    custom_image_pred=model_1(custom_image_transformed_with_batch_size)

In [ ]:
custom_image_pred

In [ ]:
print(custom_image_pred)

custom_image_pred_probs=torch.softmax(custom_image_pred,dim=1)
print(custom_image_pred_probs)

custom_image_pred_label=torch.argmax(custom_image_pred_probs,dim=1)
print(custom_image_pred_label)

In [ ]:
custom_image_pred_class=class_names[custom_image_pred_label.cpu()] # putt this on cpu otherwise error
custom_image_pred_class

In [ ]:
def pred_and_plot_image(model:torch.nn.Module,
                        image_path:str,
                        class_names:List[str]=None,
                        transform=None,
                        device:torch.device=device):
    target_image=torchvision.io.read_image(str(image_path)).type(torch.float32)
    
    target_image=target_image / 255.
    
    if transform:
        target_image=transform(target_image)
        
    model.to(device)
    
    model.eval()
    with torch.inference_mode():
        target_image=target_image.unsqueeze(dim=0)
        
        target_image_pred=model(target_image)
    
    target_image_pred_probs=torch.softmax(target_image_pred,dim=1)    
        
    target_image_pred_label=torch.argmax(target_image_pred_probs,dim=1)
    
    plt.imshow(target_image.squeeze().permute(1,2,0))
    
    if class_names:
        title=f"{class_names[target_image_pred_label.cpu()]} | {target_image_pred_probs.max().cpu()}"
    else:
        title=f"{class_names[target_image_pred_label.cpu()]} | {target_image_pred_probs.max().cpu()}"
        
    

In [ ]:
pred_and_plot_image(model_1,
                    custom_image_path,
                    class_names,
                    custom_image_transform,
                    device)

# Exercise

## 1] Our models are underperforming (not fitting the data well). What are 3 methods for preventing underfitting? Write them down and explain each with a sentence.


### 1} Add more features 
#### -> Adding more relevant features to the model can help it better capture the underlying relationships in the data.

### 2} Increase model complexity 
#### -> Using a more complex model architecture with more parameters can allow the model to fit a wider range of functions.

### 3} Reduce regularization 
#### -> Decreasing regularization techniques like L1/L2 regularization reduces restrictions on the model and allows it to fit the training data better.

## 2] Recreate the data loading functions we built in sections 1, 2, 3 and 4. You should have train and test DataLoader's ready to use.


In [22]:
import requests
import zipfile
from pathlib import Path

data_path=Path("data/")
image_path=data_path / "pizza_staek_sushi"

image_path.mkdir(parents=True,exist_ok=True)

with open(data_path / "pizza_staek_sushi.zip","wb") as f:
    request=requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    f.write(request.content)
    
with zipfile.ZipFile(data_path / "pizza_staek_sushi.zip","r") as f:
    f.extractall(image_path)

In [23]:
train_dir=image_path / "train"
test_dir=image_path / "test"

In [24]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [25]:
data_transform=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.RandomHorizontalFlip(p=0.6),
    transforms.ToTensor()
])

In [26]:
train_data=datasets.ImageFolder(root=train_dir,
                                transform=data_transform,
                                target_transform=None)
test_data=datasets.ImageFolder(root=test_dir,
                                transform=data_transform)

In [27]:
class_names=train_data.classes

In [28]:
class_dict=train_data.class_to_idx

In [29]:
BATCH_SIZE=1
NUM_WORKERS=1

train_dataloader=DataLoader(dataset=train_data,
                            batch_size=BATCH_SIZE,
                            num_workers=NUM_WORKERS,
                            shuffle=True)
test_dataloader=DataLoader(dataset=test_data,
                            batch_size=BATCH_SIZE,
                            num_workers=NUM_WORKERS,
                           shuffle=False)

## 3] Recreate model_0 we built in section 7.


In [30]:
transform=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

In [31]:
train_data_1=datasets.ImageFolder(root=train_dir,
                                  transform=transform
                                  )
test_data_1=datasets.ImageFolder(root=test_dir,
                                  transform=transform
                                  )

import os

BATCH_SIZE=32
NUM_WORKERS=os.cpu_count()

train_dataloader_1=DataLoader(train_data_1,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              shuffle=True)
test_dataloader_1=DataLoader(train_data_1,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              shuffle=False)

In [32]:
from torch import nn

In [33]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [34]:
class ModelRebuild(nn.Module):
    def __init__(self,input_shape:int,hidden_units:int, output_shape):
        super().__init__()
        self.conv_block1=nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.conv_block2=nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*15*15,out_features=output_shape)
        )
    def forward(self,x:torch.Tensor):
            x=self.conv_block1(x)
            x=self.conv_block2(x)
            print(x.shape)
            x=self.classifier(x)
            return x
        
torch.manual_seed(29)
model_e=ModelRebuild(3,10,len(train_data_1.classes)).to(device)
model_e

ModelRebuild(
  (conv_block1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2250, out_features=3, bias=True)
  )
)

## 4] Create training and testing functions for model_0.


In [35]:
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer):
    model.train()
    
    train_loss, train_acc=0,0
    
    for batch, (X,y) in enumerate(dataloader):
        X,y=X.to(device), y.to(device)
        
        y_pred=model(X)
        
        loss=loss_fn(y_pred,y)
        train_loss+=loss.item()
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
        train_acc+=(y_pred_class==y).sum().item()/len(y_pred)
    
    # adjust metrics to get average loss and accuracy pre batch
    train_loss=train_loss / len(dataloader)
    train_acc=train_acc/ len(dataloader)
    
    return train_loss, train_acc

In [36]:
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module):
    
    model.eval()
    
    test_loss, test_acc=0,0
    
    with torch.inference_mode():
        for batch, (X,y) in enumerate(dataloader):
            X,y=X.to(device), y.to(device)
            
            test_pred_logits=model(X)
            
            loss=loss_fn(test_pred_logits,y)
            test_loss+=loss.item()
            
            test_pred_labels=test_pred_logits.argmax(dim=1)
            test_acc+=((test_pred_labels==y).sum().item() / len(test_pred_labels))
            
    test_loss/=len(dataloader)
    test_acc/=len(dataloader)
    return test_loss, test_acc

In [37]:
from tqdm.auto import tqdm

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int=5):
    results={"train_loss":[],
             "train_acc":[],
             "test_loss":[],
             "test_acc":[]}
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc=train_step(model=model,
                                         dataloader=train_dataloader,
                                         loss_fn=loss_fn,
                                         optimizer=optimizer)
        test_loss, test_acc=test_step(model=model,
                                      dataloader=test_dataloader,
                                      loss_fn=loss_fn)
        
        print(
            f"epoch+1 | " 
            f"train_loss: {train_loss} | "
            f"train_acc: {train_acc} | "            
            f"test_loss: {test_loss} | "
            f"test_acc: {test_acc} "
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
    return results

## 5] Try training the model you made in exercise 3 for 5, 20 and 50 epochs, what happens to the results?
- Use torch.optim.Adam() with a learning rate of 0.001 as the optimizer.


In [38]:
torch.manual_seed(29)
torch.cuda.manual_seed(29)

NUM_EPOCHS=5

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model_e.parameters(),0.001)

from timeit import default_timer as timer
start_time=timer()

model_e_res=train(model_e,
                  train_dataloader_1,
                  test_dataloader_1,
                  optimizer,
                  loss_fn,
                  NUM_EPOCHS)

end_time=timer()
print(end_time-start_time)

  0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([1, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([1, 10, 15, 15])
epoch+1 | train_loss: 1.102908879518509 | train_acc: 0.265625 | test_loss: 1.0963498651981354 | test_acc: 0.43359375 
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([1, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32, 10, 15, 15])
torch.Size([32,

In [39]:
model_e_res["test_acc"][-1]

0.64453125

## 6] Double the number of hidden units in your model and train it for 20 epochs, what happens to the results?


In [40]:
torch.manual_seed(29)
torch.cuda.manual_seed(29)

NUM_EPOCHS=20

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model_e.parameters(),0.001)

from timeit import default_timer as timer
start_time=timer()

model_e_double_units=ModelRebuild(3,20,len(train_data_1.classes)).to(device)
model_e_double_units_res=train(model_e_double_units,
                  train_dataloader_1,
                  test_dataloader_1,
                  optimizer,
                  loss_fn,
                  NUM_EPOCHS)

end_time=timer()
print(end_time-start_time)

  0%|          | 0/20 [00:00<?, ?it/s]

torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([1, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([1, 20, 15, 15])
epoch+1 | train_loss: 1.0952823460102081 | train_acc: 0.3984375 | test_loss: 1.095036894083023 | test_acc: 0.3984375 
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([1, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32, 20, 15, 15])
torch.Size([32,

In [41]:
model_e_double_units_res["test_acc"][-1]

0.3984375

## 7] Double the data you're using with your model and train it for 20 epochs, what happens to the results?
- Note: You can use the custom data creation notebook to scale up your Food101 dataset.
- You can also find the already formatted double data (20% instead of 10% subset) dataset on GitHub, you will need to write download code like in exercise 2 to get it into this notebook.


In [46]:
data_path_1=Path("data/")
image_path_1=data_path_1 / "pizza_steak_sushi_20_percent"

image_path_1.mkdir(parents=True,exist_ok=True)

# print(requests.head("https://github.com/mrdbourke/pytorch-deep-learning/blob/main/data/pizza_steak_sushi_20_percent.zip"))

with open(data_path_1 / "pizza_steak_sushi_20_percent.zip","wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
    f.write(request.content)
    
with zipfile.ZipFile(data_path_1 / "pizza_steak_sushi_20_percent.zip","r") as f:
    f.extractall(image_path_1)

In [49]:
train_dir_1=image_path_1 / "train"
test_dir_1=image_path_1 / "test"

In [50]:
train_data_1=datasets.ImageFolder(root=train_dir_1,
                                transform=data_transform,
                                target_transform=None)
test_data_1=datasets.ImageFolder(root=test_dir_1,
                                transform=data_transform)

In [51]:
class_names_1=train_data_1.classes

In [52]:
class_dict_1=train_data_1.class_to_idx

In [56]:
train_data_1=datasets.ImageFolder(root=train_dir,
                                  transform=transform
                                  )
test_data_1=datasets.ImageFolder(root=test_dir,
                                  transform=transform
                                  )

import os

BATCH_SIZE=32
NUM_WORKERS=os.cpu_count()

train_dataloader_1=DataLoader(train_data_1,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              shuffle=True)
test_dataloader_1=DataLoader(train_data_1,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              shuffle=False)

In [57]:
model_e_double_data=ModelRebuild(3,10,len(train_data_1.classes)).to(device)
model_e_double_data

ModelRebuild(
  (conv_block1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2250, out_features=3, bias=True)
  )
)

## 8] Make a prediction on your own custom image of pizza/steak/sushi (you could even download one from the internet) and share your prediction.
- Does the model you trained in exercise 7 get it right?
- If not, what do you think you could do to improve it?